# Resume Specialist Using Reflection Technique

You want to write a resume, but you don't like writing like me! or you are not in the mood. Are you looking for a resume specialist who write a resume for you while you talk about your skills, education, experience and etc. If so, please read this post on how to create a resume specialist that generate and criticises itself until it comes up with the most optimal resume for the user.


This resume specialist uses a self reflection technique to critisize itself. For that purpose, I used LangGraph. You can read more about LangGraph here. 
In self reflection technique, LLM observes its past actions and evaluates them in order to improve the quality of output later on.
Here are the overview of the usecase:
* Generate the trasncription of the audio resume using a speech-to-text model.
* Generate the first draft of the resume
* Criticies the generated resume
* Continue until the stop condition is satisfied

<center><figure><img src="imgs/resume_specialist diagram.jpg" alt="drawing" width="1000"/><figcaption>Fig. 1: Resume specialist architecture</figcaption></figure></center>   

In [ ]:
! pip install -U --quiet  langchain langgraph fireworks-ai openai

In [15]:
import os
from getpass import getpass
os.environ['FIREWORKS_API_KEY'] = getpass('Enter your FIREWORKS API key: ')
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API Key: ')

Enter your FIREWORKS API key:  ········
Enter your OpenAI API Key:  ········


In [17]:
from langchain.prompts import ChatMessagePromptTemplate
from langchain_community.chat_models.fireworks import ChatFireworks
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, BaseMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

In [ ]:
llm = ChatFireworks(
    model="accounts/fireworks/models/mixtral-8x7b-instruct",
    model_kwargs={"max_tokens": 32768})

In [18]:
async def generate(state: Sequence[BaseMessage]):
    prompt = ChatPromptTemplate.from_messages(
        [
            "system",
            "You are a resume assistant tasked with writing excellent resumes."
            " Generate the best resume possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    
    chain = prompt | llm
    # print("###### message type: #######",messages[0].type) 
    return await chain.ainvoke({"messages": state})        

In [ ]:
async def reflect(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    reflection_prompt = ChatPromptTemplate.from_messages(
        [
            "system",
            "You are a resume reviewer evaluating a resume submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    for msg in messages:
        print(f"message type is :{msg.type}\n")
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    chain = reflection_prompt | llm
    reflection = await chain.ainvoke({"messages": translated})
    return HumanMessage(reflection.content)

In [ ]:
def should_continue(state: List[BaseMessage]):
    print("$$$$$$$ messages length: $$$$$$$$$", len(state))
    if len(state) > 6:
        # End after 3 iterations
        return END
    else:
        return "reflect"

## We define our graph as follows:
* generate node: is responsible for generating a resume based on the resume transcription of the user
* reflect node: criticizes the generated resume and gives several recommendations on it
* should_continue : is a conditional edge which decides to repeat the process of generat and review or quit the loop and output the final version
* reflect - generate: is a normal edge from reflect node to generate node which causes the LLM to revise the previous attemp and apply the new comments to the CV.

<center><figure><img src="imgs/resume_specialist.jpg" alt="drawing" width="600"/><figcaption>Fig. 1: Graph of the example</figcaption></figure></center>   

In [19]:
from typing import List, Sequence
from langgraph.graph import END, MessageGraph

graph = MessageGraph()

In [20]:
graph.add_node("generate",generate)
graph.add_node("reflect",reflect)
graph.set_entry_point("generate")


graph.add_conditional_edges("generate", should_continue)
graph.add_edge("reflect", "generate")
app = graph.compile()

In [22]:
from openai import OpenAI
client = OpenAI()

audio_file = open("docs/resume.m4a", "rb")
resume_transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file, 
  response_format="text"
)
print(resume_transcription)

My name is Joe Smith. I got my PhD degree in computer science from University of Toronto in 2017. After that, I worked for X company as a machine learning engineer. My task was to develop different machine learning models for X project. I also mentored some junior developers as well. I also deployed and monitored the models in production. After that, I joined Y company in 2022 as a senior language model researcher. My task was to conduct research on LLM models and how to fine tune and also augment them with some techniques. I also prototyped some use cases using the available large language tools. About my skills, I am proficient in Python and I also have a good experience working with deep learning frameworks such as PyTorch and TensorFlow. I also have a good experience using Panda and SQL for data manipulation and I am able to work with machine learning libraries such as Scikit-learn. If you want to reach me, my email is joe.smith at example.com



In [ ]:
input = HumanMessage(content = f"""Write a resume based on the following information: \n
                                Resume : {resume_transcription} \n
                                """)

async for event in graph.astream(input):
    print("---")

In [ ]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()